In [26]:
import numpy as np
import scipy as sps
import pandas as pd
'''
def load_data(filename="dataset/raw_source/sinwave.csv", seq_len, normalise_window):
'''

f = open("dataset/raw_source/sinwave.csv", 'rb').read()
data = f.decode().split('\n')
#sequence_length = seq_len + 1
sequence_length = 100 + 1
result = []
for index in range(len(data)-sequence_length):
    result.append(data[index: index+sequence_length])
result = np.array(result)
row = round(0.9 * result.shape[0])
print(result)
train = result[:int(row), :]
print(train)
np.random.shuffle(train)
x_train = train[:, :-1]
#print(x_train)
y_train = train[:, -1]
#print(y_train)
x_test = result[int(row):, :-1]
#print(x_test)
y_test = result[int(row):, -1]
#print(y_test)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  
print(x_train)


[['0.841470985' '0.873736397' '0.90255357' ... '0.7671179' '0.805884672'
  '0.841470985']
 ['0.873736397' '0.90255357' '0.927808777' ... '0.805884672'
  '0.841470985' '0.873736397']
 ['0.90255357' '0.927808777' '0.949402346' ... '0.841470985'
  '0.873736397' '0.90255357']
 ...
 ['0.725323664' '0.7671179' '0.805884672' ... '0.633323869' '0.680666907'
  '0.725323664']
 ['0.7671179' '0.805884672' '0.841470985' ... '0.680666907' '0.725323664'
  '0.7671179']
 ['0.805884672' '0.841470985' '0.873736397' ... '0.725323664' '0.7671179'
  '0.805884672']]
[['0.841470985' '0.873736397' '0.90255357' ... '0.7671179' '0.805884672'
  '0.841470985']
 ['0.873736397' '0.90255357' '0.927808777' ... '0.805884672'
  '0.841470985' '0.873736397']
 ['0.90255357' '0.927808777' '0.949402346' ... '0.841470985'
  '0.873736397' '0.90255357']
 ...
 ['0.991435244' '0.997679266' '0.999985904' ... '0.967249058'
  '0.98127848' '0.991435244']
 ['0.997679266' '0.999985904' '0.998346054' ... '0.98127848'
  '0.991435244' '0.

In [5]:
x_train

array([], shape=(18699, 0, 1), dtype='<U775')

In [1]:
import numpy as np
import scipy as sps
import pandas as pd

In [2]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

from utils.keras_utils import train_model, evaluate_model, set_trainable
from utils.layer_utils import AttentionLSTM

DATASET_INDEX = 14

MAX_TIMESTEPS = 291
MAX_NB_VARIABLES = 72
NB_CLASS = 2

TRAINABLE = True


def generate_model():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))

    x = Masking()(ip) # if all values in the input tensor at that timestep are equal to mask_value, then the timestep will be masked
    x = LSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model


def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor

    Returns: a keras tensor
    '''
    filters = input._keras_shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

/Users/massimo/anaconda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
model = generate_model()
train_model(model, DATASET_INDEX, dataset_prefix='ozone', epochs=600, batch_size=128)
evaluate_model(model, DATASET_INDEX, dataset_prefix='ozone', batch_size=128)
